I used Ollama to run the LLM (download from https://ollama.com/ and run exe)
pip install Ollama
ollama pull llama3.1

the rest is done in a virtual env.



In [ ]:
%pip install langchain langchain_community pypdf docarray pydantic==1.10.8

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

MODEL = 'llama3.1'
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("./docs/I-90_Instructions.pdf")
pages = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50).split_documents(loader.load())
pages

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based only on the context below.
If you can't answer the question, reply "I can't find any information about that".
Do not discuss topics outside of the context.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
questions = [
    "What is a valid signature?",
    "What is form I-751?",
    "What is a POE?",
    "Where can I check my case status?"
]

for i in questions:
    print(f"Questions: {i}")
    print(f"Answer: {chain.invoke({'question': i})}")
    print()